# Satellite Imagery

To plot satellite imagery we can use data that we bring in through our local data feed. Satellite data are stored in ```/data/ldmdata/satellite``` for both GOES-16 and GOES-17. There are a couple of different sectors that we can view the data from, but the most common for synoptic-dynamic purposes would be the CONUS projection. Data are available every 5 min.

For this plotting we are doing to use the declarative plotting syntax using Python and the MetPy module.

In [ ]:
from datetime import datetime

import metpy.calc as mpcalc
from metpy.plots import ImagePlot, ContourPlot, MapPanel, PanelContainer
from metpy.units import units
import numpy as np
from siphon.catalog import TDSCatalog
import xarray as xr

In [ ]:
date = datetime(2020, 3, 2, 0)

# Create variables for URL generation
region = 'CONUS'
channel = 9
satellite = 'east'

# We want to match something like:
# https://thredds-test.unidata.ucar.edu/thredds/catalog/satellite/goes16/GOES16/Mesoscale-1/Channel08/20181113/catalog.html

# Construct the data_url string
data_url = (f'https://thredds.ucar.edu/thredds/catalog/satellite/goes/{satellite}/products/'
            f'CloudAndMoistureImagery/{region}/Channel{channel:02d}/'
            f'{date:%Y%m%d}/catalog.xml')

cat = TDSCatalog(data_url)
dataset = cat.datasets[1]

# Open most recent file available
ds = dataset.remote_access(use_xarray=True)

# Apply a square root correction for visible imagery only
#ds = ds.map(np.sqrt, keep_attrs=True)

# Grab time from file and make into datetime object for plotting and later data access
vtime = ds.time.values.astype('datetime64[ms]').astype('O')

In [ ]:
# Declare the data we wish to plot
img = ImagePlot()
img.data = ds
img.field = 'Sectorized_CMI'
img.colormap = 'Greys'

# Plot the data on a map
panel = MapPanel()
panel.layers = ['coastline', 'borders', 'states']
panel.title = f"{'GOES-16 WV Channel 9':<123}{f'Valid Time: {vtime} UTC':>123}"
panel.plots = [img]

# Place the map on a figure
pc = PanelContainer()
pc.size = (20, 16)
pc.panels = [panel]
pc.show()